# Setup

In [1]:
# Only run once
# !pip install librosa
# !pip install tensorflow

In [2]:
# Package Imports
import pandas as pd
import numpy as np
import librosa
from scipy.io import wavfile
import soundfile as sf
import glob
import os
import boto3
import tensorflow as tf


s3 = boto3.resource('s3')


# S3 Bucket for Professor's Account is 'monitoring-whale-recordings'
# S3 Bucket for our free tier Account is 'monitoring-whale-records'
bucket_name = 'monitoring-whale-recordings'
bucket = s3.Bucket(bucket_name)


## CONSTANTS
# Window size (n_fft) in seconds
WINDOW_SIZE_SEC = 0.15175
# Hop Length in seconds
HOP_LEN_SEC = 0.05
# Decimated .wav files w/ their file path
WAV_FILES = []
for file in bucket.objects.all():
    if file.key.split('/')[-2] == 'decimated_files':
        wav = file.key.split('/')[-1] 
        path = file.key
        if wav != '':
            WAV_FILES.append([wav, path])

2023-06-16 19:52:09.262627: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 19:52:10.119991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Reading .wav Files

In [3]:
def read_wavfile(wname, verbose=False):
    """
    Reads in a decimated wav file from the S3 Bucket.
    
    PARAMETERS
    ----------
        wname: string
            Decimated WAV file's name
        verbose: boolean
            Indicates whether or not to make output excessively detailed
    ----------
    
    RETURNS
    ----------
        sr: int
            Sampling rate of WAV file
        data: numpy array
            Contains floats representing the amplitudes of the sound wave for each sample (automatically ordered from earliest to latest)
    ----------
    """
    # Reads-in the decimated WAV file's information.
    try:
        sr, data = wavfile.read(wname) # NOTE: Sampling rate (sr) seems to be 8000 samples per second
    except Exception:
        print(f'{wname} not found')
        exit(1)
        
    if verbose:
        print(f'{wname}: {data.shape[0]} samples at {samp_rate} samples/sec --> {data.shape[0]/sr} seconds')
    
    return sr, data

# PCEN Implementation

In [4]:
def process_wav(wname, verbose=False, batch_size=30, running=False):
    """
    Takes in a decimated WAV file and implements PCEN (and grabs its bounding box annotations)
    
    PARAMETERS
    ----------
        wname: string
            Decimated WAV file's name
        verbose: boolean
            Indicates whether or not to make output excessively detailed
    ----------
    
    RETURNS
    ----------
        A tensorflow dataset
    ----------
    """
    # Reads-in WAV file information
    sr, data = read_wavfile(wname, verbose=verbose)

    # Parameters needed for the stream
    n_fft = int(WINDOW_SIZE_SEC * sr)
    hop_length = int(HOP_LEN_SEC * sr)

    # Streaming processes larger audio files faster (block-wise processing)
    # note: when hop_length < frame_length, neighboring frames overlap
    stream = librosa.stream(wname, 
                            block_length=192, # num of frames per block (400 => 1 min)
                            frame_length=n_fft, # num of samples per frame
                            hop_length=hop_length, # num of samples between start of each frame
                            mono=True,
                            fill_value=0)

    # Variables to initilialize
    pcen_blocks = []
    zi = None
    S = None

    for y_block in stream:
        # Compute the STFT (without padding, so center=False)
        S = librosa.stft(y_block, n_fft=n_fft, hop_length=hop_length, 
                         center=False, out=S)
        
        # Implement PCEN on the block
        P, zi = librosa.pcen(np.abs(S), sr=sr, hop_length=hop_length,
                             gain=0.7, bias=20, power=0.2, time_constant=4, 
                             zi=zi, return_zf=True)

        pcen_blocks.append(P)
    pcen_blocks=np.array(pcen_blocks)
    pcen_blocks=np.expand_dims(pcen_blocks, axis=-1)
    if running:
        return pcen_blocks,sr
    dataset = tf.data.Dataset.from_tensor_slices(pcen_blocks)
    dataset = dataset.shuffle(buffer_size=len(pcen_blocks))
    dataset = dataset.batch(batch_size)
    return dataset,sr

# Copy Files to S3 Bucket

In [5]:
def copy_to_bucket(fileSource, fileDestination):
    """
    Copies a file from this notebook's working directory to the S3 Bucket.
    """
    # NOTE: Change the following name of the S3 Bucket (in parentheses) to reflect the name of the S3 Bucket for your current AWS account.
    """WARNING: This S3 Bucket should be the one that contains SageMaker files (NOT the one with WAV files and TXT files)."""
    write_bucket = s3.Bucket('sagemaker-us-west-2-9')
    write_bucket.upload_file(fileSource, fileDestination)